In [0]:
# Trabajamos en Databricks Community Edition: https://community.cloud.databricks.com/login.html
    # 1º: creamos el clúster
    # 2º: creamos el notebook
    
    
# Trabajaremos sobre colecciones de datos denominadas 'RDD' (son los datos en los workers, es el concepto básico de datos con el que trabaja Spark)

# Sobre los RDDs hay dos tipos de operaciones principales: TRANSFORMACIONES (día 2) y ACCIONES (día 3)
    # Transformaciones: devuelven otro RDD (evaluación perezosa/demorada)
    # Acciones: devuelven un valor (resultado)
    
    
# Evaluación perezosa/demorada:
    # En Spark, la evaluación perezosa/demorada significa que las transformaciones en un RDD (Resilient Distributed Dataset) no se evalúan   inmediatamente después de su definición, sino que se posponen hasta que se solicita una acción. Esto permite que Spark realice optimizaciones en el plan de ejecución y minimice la cantidad de datos que se deben transferir en la red.
    # Spark “va apuntando” las operaciones sobre los datos (transformaciones), no se ejecutan de forma inmediata.
    # No se calcula nada hasta que es necesario porque así se solicita expresamente.
    
    
# TRANSFORMACIONES MÁS COMUNES
    # map(func): Crea un nuevo RDD a partir de otro aplicando una  transformación a cada elemento original
    # filter(func): Crea un nuevo RDD a partir de otro manteniendo solo  los elementos de la lista original que cumplan una  condición
    # flatMap(func): Como map pero cada elemento original se puede  mapear a 0 o varios elementos de salida
    # distinct(): Crea un nuevo RDD a partir de otro eliminando  duplicados
    # union(otroRDD): Une dos RDD en uno
    # sample(): Obtiene un RDD con una muestra obtenida con reemplazamiento (o sin) a partir de otro RDD.

In [0]:
def sumar(numero1,numero2):
    suma = numero1 +numero2
    return suma

In [0]:
lambda numero1, numero2: numero1 + numero2

Out[11]: <function __main__.<lambda>(numero1, numero2)>

In [0]:
#crea un objeto RDD, en este caso: contiene los números del 1 al 10 y con dos particiones.

numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10], 2)     #sc = SparkContext

In [0]:
#recupera todos los elementos de un RDD como una lista en el programa de control (evaluación perezosa)

numeros.collect()

Out[13]: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [0]:
#map() aplica una transformación a cada elemento del RDD original

numeros = sc.parallelize([1,2,3,4,5])

num3 = numeros.map(lambda elemento: 3*elemento)

num3.collect()

Out[14]: [3, 6, 9, 12, 15]

In [0]:
palabras = sc.parallelize(['HOLA', 'Que', 'TAL', 'Bien'])

pal_minus = palabras.map(lambda elemento: elemento.lower())

pal_minus.collect()

Out[15]: ['hola', 'que', 'tal', 'bien']

In [0]:
#filter() filtra un RDD manteniendo solo los elementos que cumplan  una determinada condición

numeros = sc.parallelize([1,2,3,4,5])

pares_rdd = numeros.filter(lambda elemento: elemento%2==0)

pares_rdd.collect()

Out[18]: [2, 4]

In [0]:
log = sc.parallelize(['E: e21', 'W: w12', 'W: w13', 'E: e45'])

errors = log.filter(lambda elemento: elemento[0]=='E')

errors.collect()

Out[19]: ['E: e21', 'E: e45']

In [0]:
log = sc.parallelize(['E: e21', 'W: w12', 'W: w13', 'E: e45'])

errors = log.filter(lambda elemento: elemento[-2:] == '13')
#errors = log.filter(lambda elemento: elemento.endswith('13'))
errors.collect()

Out[39]: ['W: w13']

In [0]:
# flatMap() devuelve el resultado “plano”, sin diferenciar la salida de cada elemento (corchetes, etc). Útil cuando se utilizan funciones que devuelven listas, tuplas…

numeros = sc.parallelize([1,2,3,4,5])

rdd = numeros.flatMap(lambda elemento : [elemento, 10*elemento])

rdd.collect()

Out[44]: [1, 10, 2, 20, 3, 30, 4, 40, 5, 50]

In [0]:
#Diferencia map y flatMap

lineas = sc.parallelize(['', 'a', 'a b', 'a b c'])

palabras_map = lineas.map(lambda elemento: elemento.split())     #espacios en blanco como separadores (solo dentro de comillas) 

palabras_flat = lineas.flatMap(lambda elemento: elemento.split())    #igual, pero al aplanar, separo todas

print("RDD ORIGINAL: ", lineas.collect())
print("RDD resultado MAP: ", palabras_map.collect())
print("RDD resultado FLATMAP: ", palabras_flat.collect())

RDD ORIGINAL:  ['', 'a', 'a b', 'a b c']
RDD resultado MAP:  [[], ['a'], ['a', 'b'], ['a', 'b', 'c']]
RDD resultado FLATMAP:  ['a', 'a', 'b', 'a', 'b', 'c']


In [0]:
# distinct() nuevo RDD eliminando duplicados

numeros = sc.parallelize([1,1,2,2,5])

unicos = numeros.distinct()

unicos.collect()

Out[56]: [1, 2, 5]

In [0]:
# sample() devuelve una muestra del RDD en ateción al reemplazamiento (True/False) y fracción que se indican (0-1)

numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

muestra = numeros.sample(False, 0.5)    #False = no repetidos (no se reemplazan) // se seleccionará al azar, aprox, el 50% de los elementos

muestra.collect()

Out[75]: [2, 3, 4, 5, 6, 9, 10]

In [0]:
# union() une dos RDDs en uno

pares = sc.parallelize([2,4,6,8,10])
impares = sc.parallelize([1,3,5,7,932232])

numeros = pares.union(impares)

numeros.collect()

Out[77]: [2, 4, 6, 8, 10, 1, 3, 5, 7, 932232]

In [0]:
# Une las dos cadenas creadas

log = sc.parallelize(['E: e21', 'I: i11', 'W: w12', 'I: i11', 'W: w13', 'E: e45'])
log.setName("archivo log original")

info = log.filter(lambda elemento: elemento[0]=='I')
info.setName("INFO filtrado")

error = log.filter(lambda elemento: elemento[0]=='E')
error.setName("ERROR filtrado")

inferr = info.union(error)
inferr.setName("UNION filtrados")

inferr.collect()

Out[80]: ['I: i11', 'I: i11', 'E: e21', 'E: e45']